# Monet Style GAN - DCGAN Implementation

This notebook implements a Deep Convolutional GAN (DCGAN) to generate Monet-style paintings.

**Competition:** GAN Getting Started - Kaggle  
**Team:** Azmi Abidi, Guerlain Hitier-Lallement, Kaothar Reda  

**Evaluation:** FID (Fréchet Inception Distance) score

## 1. Setup and Imports

In [ ]:
import os
import tempfile
import shutil
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.utils as vutils

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Configuration

In [ ]:
# Paths
DATA_DIR = '/kaggle/input/gan-getting-started'  
OUTPUT_DIR = '/kaggle/working/outputs'

# Create directories
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Hyperparameters - Aggressively optimized for better FID
IMG_SIZE = 256
BATCH_SIZE = 4           # Smaller batches = more stable training
NZ = 256                 # Much larger latent space for diversity
NGF = 96                 # Increased generator capacity
NDF = 48                 # Reduced discriminator capacity (prevent overpowering)
NC = 3
EPOCHS = 75              # As requested
LR_G = 3e-4              # Higher generator LR
LR_D = 5e-5              # Much lower discriminator LR (key change!)
BETA1 = 0.5
BETA2 = 0.999
N_CRITIC = 1             # Equal training

print(f"Configuration:")
print(f"  Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Generator LR: {LR_G} (capacity: {NGF})")
print(f"  Discriminator LR: {LR_D} (capacity: {NDF})")
print(f"  Latent Dimension: {NZ}")
print(f"  LR Ratio (G/D): {LR_G/LR_D:.1f}x")

## 3. Dataset Loader

In [ ]:
class MonetDataset(Dataset):
    """Dataset for loading Monet paintings with strong data augmentation."""
    
    def __init__(self, data_dir, img_size=256, transform=None, augment=True):
        self.data_dir = Path(data_dir)
        self.img_size = img_size
        self.augment = augment
        
        # Find monet images
        self.monet_dir = self.data_dir / 'monet_jpg'
        if not self.monet_dir.exists():
            raise FileNotFoundError(f"Directory {self.monet_dir} not found")
        
        self.image_paths = sorted(list(self.monet_dir.glob('*.jpg')))
        
        if len(self.image_paths) == 0:
            raise ValueError(f"No images found in {self.monet_dir}")
        
        print(f"Found {len(self.image_paths)} Monet paintings")
        
        # Strong augmentation to prevent memorization and increase diversity
        if transform is None:
            if augment:
                self.transform = transforms.Compose([
                    transforms.Resize((int(img_size * 1.1), int(img_size * 1.1))),  # Slightly larger
                    transforms.RandomCrop((img_size, img_size)),  # Random crop
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.RandomRotation(degrees=5),  # Slight rotation
                    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.03),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
                ])
            else:
                self.transform = transforms.Compose([
                    transforms.Resize((img_size, img_size)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
                ])
        else:
            self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image

# Create dataset and dataloader with strong augmentation
dataset = MonetDataset(DATA_DIR, img_size=IMG_SIZE, augment=True)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, 
                       num_workers=2, pin_memory=True, drop_last=True)

print(f"✓ Dataset loaded with strong augmentation (crop, flip, rotate, color jitter)")

## 4. Model Architecture

In [ ]:
def _cap(ch):
    """Cap channels at 512 to avoid excessive memory usage."""
    return min(ch, 512)

class Generator(nn.Module):
    """DCGAN Generator for 256x256 images with increased capacity."""
    
    def __init__(self, nz=256, ngf=96, nc=3):
        super(Generator, self).__init__()
        self.nz = nz
        
        self.main = nn.Sequential(
            # Input: nz x 1 x 1
            nn.ConvTranspose2d(nz, _cap(ngf * 16), 4, 1, 0, bias=False),
            nn.BatchNorm2d(_cap(ngf * 16)),
            nn.ReLU(True),
            # State: (ngf*16) x 4 x 4
            nn.ConvTranspose2d(_cap(ngf * 16), _cap(ngf * 8), 4, 2, 1, bias=False),
            nn.BatchNorm2d(_cap(ngf * 8)),
            nn.ReLU(True),
            # State: (ngf*8) x 8 x 8
            nn.ConvTranspose2d(_cap(ngf * 8), _cap(ngf * 4), 4, 2, 1, bias=False),
            nn.BatchNorm2d(_cap(ngf * 4)),
            nn.ReLU(True),
            # State: (ngf*4) x 16 x 16
            nn.ConvTranspose2d(_cap(ngf * 4), _cap(ngf * 2), 4, 2, 1, bias=False),
            nn.BatchNorm2d(_cap(ngf * 2)),
            nn.ReLU(True),
            # State: (ngf*2) x 32 x 32
            nn.ConvTranspose2d(_cap(ngf * 2), _cap(ngf), 4, 2, 1, bias=False),
            nn.BatchNorm2d(_cap(ngf)),
            nn.ReLU(True),
            # State: ngf x 64 x 64
            nn.ConvTranspose2d(_cap(ngf), _cap(ngf // 2), 4, 2, 1, bias=False),
            nn.BatchNorm2d(_cap(ngf // 2)),
            nn.ReLU(True),
            # State: (ngf//2) x 128 x 128
            nn.ConvTranspose2d(_cap(ngf // 2), nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output: nc x 256 x 256
        )
    
    def forward(self, z):
        # Reshape input if needed
        if z.dim() == 2:
            z = z.view(z.size(0), z.size(1), 1, 1)
        return self.main(z)


class Discriminator(nn.Module):
    """DCGAN Discriminator for 256x256 images with reduced capacity and dropout."""
    
    def __init__(self, nc=3, ndf=48):
        super(Discriminator, self).__init__()
        
        self.main = nn.Sequential(
            # Input: nc x 256 x 256
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.3),  # Add dropout for regularization
            # State: ndf x 128 x 128
            nn.Conv2d(ndf, _cap(ndf * 2), 4, 2, 1, bias=False),
            nn.BatchNorm2d(_cap(ndf * 2)),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.3),
            # State: (ndf*2) x 64 x 64
            nn.Conv2d(_cap(ndf * 2), _cap(ndf * 4), 4, 2, 1, bias=False),
            nn.BatchNorm2d(_cap(ndf * 4)),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.3),
            # State: (ndf*4) x 32 x 32
            nn.Conv2d(_cap(ndf * 4), _cap(ndf * 8), 4, 2, 1, bias=False),
            nn.BatchNorm2d(_cap(ndf * 8)),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf*8) x 16 x 16
            nn.Conv2d(_cap(ndf * 8), _cap(ndf * 16), 4, 2, 1, bias=False),
            nn.BatchNorm2d(_cap(ndf * 16)),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf*16) x 8 x 8
            nn.Conv2d(_cap(ndf * 16), _cap(ndf * 32), 4, 2, 1, bias=False),
            nn.BatchNorm2d(_cap(ndf * 32)),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf*32) x 4 x 4
            nn.Conv2d(_cap(ndf * 32), 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
            # Output: 1 x 1 x 1
        )
    
    def forward(self, x):
        output = self.main(x)
        return output.view(-1, 1).squeeze(1)


def weights_init(m):
    """Custom weight initialization with better scaling."""
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


# Initialize models
netG = Generator(nz=NZ, ngf=NGF, nc=NC).to(device)
netD = Discriminator(nc=NC, ndf=NDF).to(device)

# Apply weight initialization
netG.apply(weights_init)
netD.apply(weights_init)

print("Generator:")
print(netG)
print(f"\nGenerator parameters: {sum(p.numel() for p in netG.parameters()):,}")
print(f"Discriminator parameters: {sum(p.numel() for p in netD.parameters()):,}")
print(f"Capacity ratio (G/D): {sum(p.numel() for p in netG.parameters())/sum(p.numel() for p in netD.parameters()):.2f}x")

## 5. Training Setup

In [ ]:
# Loss function
criterion = nn.BCELoss()

# Optimizers with aggressive LR difference
optimizerD = optim.Adam(netD.parameters(), lr=LR_D, betas=(BETA1, BETA2))
optimizerG = optim.Adam(netG.parameters(), lr=LR_G, betas=(BETA1, BETA2))

# Cosine annealing scheduler for smooth LR decay
schedulerD = optim.lr_scheduler.CosineAnnealingLR(optimizerD, T_max=EPOCHS, eta_min=1e-6)
schedulerG = optim.lr_scheduler.CosineAnnealingLR(optimizerG, T_max=EPOCHS, eta_min=1e-5)

# Fixed noise for visualization
fixed_noise = torch.randn(16, NZ, 1, 1, device=device)

# Labels with strong one-sided label smoothing
real_label = 0.85  # Smoother real labels
fake_label = 0.0

# Training history
G_losses = []
D_losses = []
D_x_history = []
D_G_z_history = []

print("Training setup complete!")
print(f"  Generator LR: {LR_G} → {1e-5}")
print(f"  Discriminator LR: {LR_D} → {1e-6}")
print(f"  Using Cosine Annealing LR scheduling")
print(f"  LR advantage for Generator: {LR_G/LR_D:.1f}x")

## 6. Training Loop

In [ ]:
print("Starting Training...\n")
print("Strategy: Strong Generator, Weak Discriminator + Heavy Augmentation\n")

for epoch in range(EPOCHS):
    epoch_D_loss = 0.0
    epoch_G_loss = 0.0
    epoch_D_x = 0.0
    epoch_D_G_z = 0.0
    
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    
    for i, real_imgs in enumerate(progress_bar):
        real_imgs = real_imgs.to(device)
        batch_size = real_imgs.size(0)
        
        # ==================== Train Discriminator ====================
        netD.zero_grad()
        
        # Train with real images
        label = torch.full((batch_size,), real_label, dtype=torch.float, device=device)
        # Noisy labels (10% flip rate for stability)
        if np.random.random() < 0.1:
            label.fill_(fake_label)
        
        output_real = netD(real_imgs)
        errD_real = criterion(output_real, label)
        errD_real.backward()
        D_x = output_real.mean().item()
        
        # Train with fake images
        noise = torch.randn(batch_size, NZ, 1, 1, device=device)
        fake_imgs = netG(noise)
        label.fill_(fake_label)
        # Noisy labels
        if np.random.random() < 0.1:
            label.fill_(real_label)
        
        output_fake = netD(fake_imgs.detach())
        errD_fake = criterion(output_fake, label)
        errD_fake.backward()
        D_G_z1 = output_fake.mean().item()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(netD.parameters(), max_norm=1.0)
        
        errD = errD_real + errD_fake
        optimizerD.step()
        
        # ==================== Train Generator (2x per D update) ====================
        for _ in range(2):  # Train G twice to help it catch up
            netG.zero_grad()
            label.fill_(real_label)
            output = netD(fake_imgs)
            errG = criterion(output, label)
            errG.backward()
            D_G_z2 = output.mean().item()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(netG.parameters(), max_norm=1.0)
            
            optimizerG.step()
            
            # Generate new fakes for second G update
            if _ == 0:
                noise = torch.randn(batch_size, NZ, 1, 1, device=device)
                fake_imgs = netG(noise)
        
        # Track losses
        epoch_D_loss += errD.item()
        epoch_G_loss += errG.item()
        epoch_D_x += D_x
        epoch_D_G_z += D_G_z2
        
        # Update progress bar
        progress_bar.set_postfix({
            'D_loss': f'{errD.item():.4f}',
            'G_loss': f'{errG.item():.4f}',
            'D(x)': f'{D_x:.3f}',
            'D(G(z))': f'{D_G_z2:.3f}'
        })
    
    # Average losses for epoch
    avg_D_loss = epoch_D_loss / len(dataloader)
    avg_G_loss = epoch_G_loss / len(dataloader)
    avg_D_x = epoch_D_x / len(dataloader)
    avg_D_G_z = epoch_D_G_z / len(dataloader)
    
    G_losses.append(avg_G_loss)
    D_losses.append(avg_D_loss)
    D_x_history.append(avg_D_x)
    D_G_z_history.append(avg_D_G_z)
    
    # Step learning rate schedulers
    schedulerD.step()
    schedulerG.step()
    
    # Print epoch summary
    if (epoch + 1) % 10 == 0:
        print(f"\nEpoch [{epoch+1}/{EPOCHS}] Summary:")
        print(f"  G_loss: {avg_G_loss:.4f}, D_loss: {avg_D_loss:.4f}")
        print(f"  D(x): {avg_D_x:.3f}, D(G(z)): {avg_D_G_z:.3f}")
        print(f"  G_LR: {schedulerG.get_last_lr()[0]:.6f}, D_LR: {schedulerD.get_last_lr()[0]:.6f}\n")
    
    # Generate and save sample images
    if (epoch + 1) % 10 == 0 or epoch == 0:
        with torch.no_grad():
            fake_samples = netG(fixed_noise).detach().cpu()
        
        # Save grid
        img_grid = vutils.make_grid(fake_samples, nrow=4, padding=2, normalize=True)
        plt.figure(figsize=(10, 10))
        plt.imshow(img_grid.permute(1, 2, 0))
        plt.axis('off')
        plt.title(f'Epoch {epoch+1}')
        plt.savefig(f'{OUTPUT_DIR}/epoch_{epoch+1:03d}.png', bbox_inches='tight', dpi=100)
        plt.close()
    
    # Save checkpoint
    if (epoch + 1) % 25 == 0 or (epoch + 1) == EPOCHS:
        torch.save({
            'epoch': epoch + 1,
            'generator': netG.state_dict(),
            'discriminator': netD.state_dict(),
            'optimizerG': optimizerG.state_dict(),
            'optimizerD': optimizerD.state_dict(),
            'schedulerG': schedulerG.state_dict(),
            'schedulerD': schedulerD.state_dict(),
            'G_losses': G_losses,
            'D_losses': D_losses,
        }, f'{OUTPUT_DIR}/checkpoint_epoch_{epoch+1:03d}.pt')
    
    # Clear CUDA cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("\nTraining Complete!")

## 7. Training Visualization

## 8. FID Evaluation

**FID (Fréchet Inception Distance)** measures the quality of generated images.  
Lower FID = better quality and more realistic images.

In [ ]:
# Install torch-fidelity for FID computation
import subprocess
import sys

try:
    import torch_fidelity
    print("✓ torch-fidelity already installed")
except ImportError:
    print("Installing torch-fidelity...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'torch-fidelity'])
    print("✓ torch-fidelity installed")
    
# Now import torchmetrics FID
try:
    from torchmetrics.image.fid import FrechetInceptionDistance
    print("✓ FID evaluation module loaded successfully")
except Exception as e:
    print(f"⚠️  Could not load FID module: {e}")
    print("FID evaluation will be skipped")

In [ ]:
def compute_fid_score_alternative(generator, num_fake_samples=1000, batch_size=16, device='cuda'):
    """
    Alternative FID computation using torch-fidelity directly.
    Saves images to temp directories and computes FID.
    """
    import tempfile
    import shutil
    
    print(f"Computing FID score with {num_fake_samples} generated samples...\n")
    
    # Create temporary directories
    temp_dir = tempfile.mkdtemp()
    real_dir = os.path.join(temp_dir, 'real')
    fake_dir = os.path.join(temp_dir, 'fake')
    os.makedirs(real_dir, exist_ok=True)
    os.makedirs(fake_dir, exist_ok=True)
    
    try:
        # Copy real images
        print("Preparing real Monet images...")
        monet_dir = Path(DATA_DIR) / 'monet_jpg'
        real_images = list(monet_dir.glob('*.jpg'))
        
        for i, img_path in enumerate(tqdm(real_images[:num_fake_samples], desc="Copying real images")):
            shutil.copy(img_path, os.path.join(real_dir, f'real_{i:05d}.jpg'))
        
        # Generate and save fake images
        print("\nGenerating fake images...")
        generator.eval()
        
        num_batches = (num_fake_samples + batch_size - 1) // batch_size
        img_counter = 0
        
        with torch.no_grad():
            for _ in tqdm(range(num_batches), desc="Generating fake images"):
                current_batch_size = min(batch_size, num_fake_samples - img_counter)
                
                # Generate images
                noise = torch.randn(current_batch_size, NZ, 1, 1, device=device)
                fake_imgs = generator(noise)
                
                # Save each image
                for i in range(current_batch_size):
                    # Denormalize from [-1, 1] to [0, 1]
                    img = (fake_imgs[i] + 1) / 2.0
                    img = img.clamp(0, 1)
                    
                    # Convert to PIL Image and save
                    img_pil = transforms.ToPILImage()(img.cpu())
                    img_pil.save(os.path.join(fake_dir, f'fake_{img_counter:05d}.jpg'))
                    img_counter += 1
        
        # Compute FID using torch-fidelity
        print("\nComputing FID score...")
        try:
            import torch_fidelity
            
            metrics = torch_fidelity.calculate_metrics(
                input1=fake_dir,
                input2=real_dir,
                cuda=torch.cuda.is_available(),
                fid=True,
                verbose=False
            )
            
            fid_score = metrics['frechet_inception_distance']
            
            print(f"\n{'='*60}")
            print(f"FID Score: {fid_score:.2f}")
            print(f"{'='*60}")
            print("\nFID Interpretation (for 256x256 GANs on small datasets):")
            print("  < 100:    Excellent - Publication quality")
            print("  100-150:  Very Good - Strong results")
            print("  150-200:  Good - Acceptable quality")
            print("  200-250:  Fair - Needs improvement")
            print("  > 250:    Poor - Significant issues")
            print(f"{'='*60}\n")
            
            return fid_score
            
        except ImportError:
            print("⚠️  torch-fidelity not available. Cannot compute FID.")
            return None
    
    finally:
        # Cleanup temp directories
        shutil.rmtree(temp_dir, ignore_errors=True)


# Compute FID score
fid_score = None
try:
    fid_score = compute_fid_score_alternative(
        generator=netG,
        num_fake_samples=1000,
        batch_size=BATCH_SIZE,
        device=device
    )
    
    if fid_score is not None:
        # Store for later reference
        evaluation_metrics = {
            'fid_score': fid_score,
            'num_samples': 1000,
            'epoch': EPOCHS
        }
    
except Exception as e:
    print(f"⚠️  FID computation failed: {e}")
    print("Continuing without FID evaluation...")
    fid_score = None

## 9. Visual Quality Assessment

In [ ]:
# Real vs Generated comparison
fig, axes = plt.subplots(2, 8, figsize=(20, 6))

# Get 8 real images
for i in range(8):
    img = Image.open(dataset.image_paths[i]).convert('RGB')
    img = transforms.Resize((256, 256))(img)
    axes[0, i].imshow(img)
    axes[0, i].set_title('Real', fontsize=10)
    axes[0, i].axis('off')

# Generate 8 fake images
netG.eval()
with torch.no_grad():
    noise = torch.randn(8, NZ, 1, 1, device=device)
    fake_imgs = netG(noise).cpu()
    fake_imgs = (fake_imgs + 1) / 2.0

    for i in range(8):
        axes[1, i].imshow(fake_imgs[i].permute(1, 2, 0).numpy())
        axes[1, i].set_title('Generated', fontsize=10)
        axes[1, i].axis('off')

plt.suptitle('Real Monet Paintings vs Generated Images', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/real_vs_generated.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Final Summary

In [ ]:
# Diversity check - generate 24 random samples
fig, axes = plt.subplots(4, 6, figsize=(18, 12))

netG.eval()
with torch.no_grad():
    for idx, ax in enumerate(axes.flat):
        noise = torch.randn(1, NZ, 1, 1, device=device)
        fake_img = netG(noise).cpu()
        fake_img = (fake_img + 1) / 2.0
        
        ax.imshow(fake_img[0].permute(1, 2, 0).numpy())
        ax.axis('off')

plt.suptitle('Diversity Check: 24 Random Samples', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/diversity_check.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
print("="*70)
print("DCGAN TRAINING SUMMARY")
print("="*70)

print(f"\n📊 Model Architecture:")
print(f"   Generator parameters: {sum(p.numel() for p in netG.parameters()):,}")
print(f"   Discriminator parameters: {sum(p.numel() for p in netD.parameters()):,}")

print(f"\n📈 Training:")
print(f"   Dataset: {len(dataset)} Monet paintings")
print(f"   Epochs: {EPOCHS}")
print(f"   Final Generator Loss: {G_losses[-1]:.4f}")
print(f"   Final Discriminator Loss: {D_losses[-1]:.4f}")

if fid_score is not None:
    print(f"\n🎯 FID Score: {fid_score:.2f}")
    
    # Realistic thresholds for 256x256 GANs on small datasets
    if fid_score < 100:
        quality = "Excellent ✓✓✓"
        emoji = "🟢"
    elif fid_score < 150:
        quality = "Very Good ✓✓"
        emoji = "🟢"
    elif fid_score < 200:
        quality = "Good ✓"
        emoji = "🟡"
    elif fid_score < 250:
        quality = "Fair"
        emoji = "🟠"
    else:
        quality = "Needs Improvement"
        emoji = "🔴"
    
    print(f"   Quality: {emoji} {quality}")
    print(f"\n   Realistic Thresholds (256x256, small dataset):")
    print(f"     < 100:    Excellent")
    print(f"     100-150:  Very Good")
    print(f"     150-200:  Good")
    print(f"     200-250:  Fair")
    print(f"     > 250:    Poor")
else:
    print(f"\n⚠️  FID Score: Not computed")

print("\n" + "="*70)

In [ ]:
# Plot training losses
plt.figure(figsize=(12, 5))
plt.plot(G_losses, label='Generator Loss', alpha=0.7)
plt.plot(D_losses, label='Discriminator Loss', alpha=0.7)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Losses')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig(f'{OUTPUT_DIR}/training_losses.png', dpi=150, bbox_inches='tight')
plt.show()

# Display final generated samples
with torch.no_grad():
    final_samples = netG(fixed_noise).detach().cpu()

fig, axes = plt.subplots(4, 4, figsize=(12, 12))
for idx, ax in enumerate(axes.flat):
    img = final_samples[idx].permute(1, 2, 0)
    img = (img + 1) / 2  # Denormalize from [-1, 1] to [0, 1]
    ax.imshow(img)
    ax.axis('off')
plt.suptitle('Final Generated Monet-Style Paintings', fontsize=16)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/final_samples.png', dpi=150, bbox_inches='tight')
plt.show()